# Getting Started

In this tutorial, you will know how to
- use the models in **ConvLab-2** to build a dialog agent.
- build a simulator to chat with the agent and evaluate the performance.
- try different module combinations.
- use analysis tool to diagnose your system.

Let's get started!

## Environment setup
Run the command below to install ConvLab-2. Then restart the notebook and skip this commend.

In [ ]:
# first install ConvLab-2 and restart the notebook
! git clone https://github.com/thu-coai/ConvLab-2.git && cd ConvLab-2 && pip install -e .

## build an agent

We use the models adapted on [Multiwoz](https://www.aclweb.org/anthology/D18-1547)  dataset to build our agent. This pipeline agent consists of NLU, DST, Policy and NLG modules.

First, import some models:

In [1]:
# common import: convlab2.$module.$model.$dataset
from convlab2.nlu.jointBERT.multiwoz import BERTNLU
from convlab2.nlu.milu.multiwoz import MILU
from convlab2.dst.rule.multiwoz import RuleDST
from convlab2.policy.rule.multiwoz import RulePolicy
from convlab2.nlg.template.multiwoz import TemplateNLG
from convlab2.dialog_agent import PipelineAgent, BiSession
from convlab2.evaluator.multiwoz_eval import MultiWozEvaluator
from pprint import pprint
import random
import numpy as np
import torch

Then, create the models and build an agent:

In [22]:
# go to README.md of each model for more information
# BERT nlu
sys_nlu = BERTNLU()
# simple rule DST
sys_dst = RuleDST()
# rule policy
sys_policy = PPOPolicy()
# template NLG
sys_nlg = TemplateNLG(is_user=False)
# assemble
sys_agent = PipelineAgent(sys_nlu, sys_dst, sys_policy, sys_nlg, name='sys')

08/02/2020 18:32:38 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): s3.amazonaws.com:443
08/02/2020 18:32:39 - DEBUG - urllib3.connectionpool -   https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/bert-base-uncased-vocab.txt HTTP/1.1" 200 0
08/02/2020 18:32:39 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/syseo/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
08/02/2020 18:32:39 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): s3.amazonaws.com:443
intent num: 137
tag num: 331
Load from /home/syseo/workspace/ConvLab-2/convlab2/nlu/jointBERT/multiwoz/output/all_context/pytorch_model.bin
bert-base-uncased
08/02/2020 18:32:40 - DEBUG - urllib3.connectionpool -   https://s3.amazonaws.com:443 "HEAD /models.hugging

That's all! Let's chat with the agent using its response function:

In [23]:
sys_agent.response("I want to find a moderate hotel")

'The address is Parkside, Cambridge . The postcode is none .'

In [24]:
sys_agent.response("Which type of hotel is it ?")

'The phone is 01223358966 . Did you need any further assistance today ?'

In [25]:
sys_agent.response("OK , where is its address ?")

'The address is Parkside, Cambridge . The phone is 01223358966 . The postcode is none .'

In [26]:
sys_agent.response("Thank you !")

'Is there anything else I can help you with today ? You are welcome.'

In [27]:
sys_agent.response("Try to find me a Chinese restaurant in south area .")

'It is located in the south . I have 3 options for you !. It is chinese food . This is a expensive one . How about the good luck chinese food takeaway ?'

In [28]:
sys_agent.response("Which kind of food it provides ?")

'It is located in the south . Would you like to try the good luck chinese food takeaway ? Is there anything else I can help you with ?'

In [29]:
sys_agent.response("Book a table for 5 , this Sunday .")

"I ' m sorry , I was unable to reserve rooms . Would you like to try anything else ? What day would you like to book ? Do you have a time preference ? Did you need any further assistance today ?"

## Build a simulator to chat with the agent and evaluate

In many one-to-one task-oriented dialog system, a simulator is essential to train an RL agent. In our framework, we doesn't distinguish user or system. All speakers are **agents**. The simulator is also an agent, with specific policy inside for accomplishing the user goal.

We use `Agenda` policy for the simulator, this policy requires dialog act input, which means we should set DST argument of `PipelineAgent` to None. Then the `PipelineAgent` will pass dialog act to policy directly. Refer to `PipelineAgent` doc for more details.

In [32]:
# MILU
user_nlu = MILU()
# not use dst
user_dst = None
# rule policy
user_policy = RulePolicy(character='usr')
# template NLG
user_nlg = TemplateNLG(is_user=True)
# user_nlg = SCLSTM(is_user=True)
# assemble
user_agent = PipelineAgent(user_nlu, user_dst, user_policy, user_nlg, name='user')

': 0.5, 'hidden_size': 200, 'input_size': 400, 'num_layers': 1, 'type': 'lstm'}, 'label_encoding': 'BIO', 'regularizer': [['scalar_parameters', {'alpha': 0.1, 'type': 'l2'}]], 'text_field_embedder': {'token_embedders': {'token_characters': {'embedding': {'embedding_dim': 16}, 'encoder': {'conv_layer_activation': 'relu', 'embedding_dim': 16, 'ngram_filter_sizes': [3], 'num_filters': 128, 'type': 'cnn'}, 'type': 'character_encoding'}, 'tokens': {'embedding_dim': 50, 'trainable': True, 'type': 'embedding'}}}, 'type': 'milu'} and extras {'vocab'}
08/02/2020 18:36:49 - INFO - allennlp.common.params -   model.type = milu
08/02/2020 18:36:49 - INFO - allennlp.common.from_params -   instantiating class <class 'convlab2.nlu.milu.model.MILU'> from params {'attention': {'matrix_dim': 400, 'type': 'bilinear', 'vector_dim': 400}, 'attention_for_intent': False, 'attention_for_tag': False, 'context_for_intent': True, 'context_for_tag': False, 'dropout': 0.3, 'encoder': {'bidirectional': True, 'dropou


Now we have a simulator and an agent. we will use an existed simple one-to-one conversation controller BiSession, you can also define your own Session class for your special need.

We add `MultiWozEvaluator` to evaluate the performance. It uses the parsed dialog act input and policy output dialog act to calculate **inform f1**, **book rate**, and whether the task is **success**.

In [11]:
evaluator = MultiWozEvaluator()
sess = BiSession(sys_agent=sys_agent, user_agent=user_agent, kb_query=None, evaluator=evaluator)

Let's make this two agents chat! The key is `next_turn` method of `BiSession` class.

In [12]:
def set_seed(r_seed):
    random.seed(r_seed)
    np.random.seed(r_seed)
    torch.manual_seed(r_seed)

set_seed(20200131)

sys_response = ''
sess.init_session()
print('init goal:')
pprint(sess.evaluator.goal)
print('-'*50)
for i in range(20):
    sys_response, user_response, session_over, reward = sess.next_turn(sys_response)
    print('user:', user_response)
    print('sys:', sys_response)
    print()
    if session_over is True:
        break
print('task success:', sess.evaluator.task_success())
print('book rate:', sess.evaluator.book_rate())
print('inform precision/recall/f1:', sess.evaluator.inform_F1())
print('-'*50)
print('final goal:')
pprint(sess.evaluator.goal)
print('='*100)

init goal:
{'restaurant': {'info': {'area': 'west', 'pricerange': 'moderate'},
                'reqt': {'phone': '?'}},
 'train': {'info': {'arriveBy': '10:45',
                    'day': 'sunday',
                    'departure': 'norwich',
                    'destination': 'cambridge'},
           'reqt': {'price': '?'}}}
--------------------------------------------------
user: I need to find information about a certain restaurant , can you help with that ? I am looking for a moderate restaurant . Is that located in the west ?
sys: I recommend saint johns chop house. Okay , may I suggest british food ? Yes there are 3 different places that match your description .

user: What is the phone number ?
sys: It is 01799521260 .

user: I also need a train. I need to find a train to cambridge please . I ' m departing from norwich . I would like to leave on sunday .
sys: When would you like to leave by ? Is there a time you need to arrive by ?

user: Well I want to arrive by 10:45 . I don't 

## Try different module combinations

The combination modes of pipeline agent modules are flexible. We support joint models such as MDBT, TRADE, SUMBT for word-DST and MDRG, HDSA, LaRL for word-Policy, once the input and output are matched with previous and next module. We also support End2End models such as Sequicity.

Available models:

- NLU: BERTNLU, MILU, SVMNLU
- DST: RuleDST
- Word-DST: SUMBT, TRADE, MDBT (set `sys_nlu` to `None`)
- Policy: RulePolicy, Imitation, REINFORCE, PPO, GDPL
- Word-Policy: MDRG, HDSA, LaRL (set `sys_nlg` to `None`)
- NLG: Template, SCLSTM
- End2End: Sequicity, DAMD, RNN_rollout (directly used as `sys_agent`)
- Simulator policy: Agenda, VHUS (for `user_policy`)


In [13]:
# available NLU models
from convlab2.nlu.svm.multiwoz import SVMNLU
from convlab2.nlu.jointBERT.multiwoz import BERTNLU
from convlab2.nlu.milu.multiwoz import MILU
# available DST models
from convlab2.dst.rule.multiwoz import RuleDST
from convlab2.dst.mdbt.multiwoz import MDBT
from convlab2.dst.sumbt.multiwoz import SUMBT
from convlab2.dst.trade.multiwoz import TRADE
# available Policy models
from convlab2.policy.rule.multiwoz import RulePolicy
from convlab2.policy.ppo.multiwoz import PPOPolicy
from convlab2.policy.pg.multiwoz import PGPolicy
from convlab2.policy.mle.multiwoz import MLEPolicy
from convlab2.policy.gdpl.multiwoz import GDPLPolicy
from convlab2.policy.vhus.multiwoz import UserPolicyVHUS
from convlab2.policy.mdrg.multiwoz import MDRGWordPolicy
from convlab2.policy.hdsa.multiwoz import HDSA
from convlab2.policy.larl.multiwoz import LaRL
# available NLG models
from convlab2.nlg.template.multiwoz import TemplateNLG
from convlab2.nlg.sclstm.multiwoz import SCLSTM
# available E2E models
from convlab2.e2e.sequicity.multiwoz import Sequicity
from convlab2.e2e.damd.multiwoz import Damd


Load from https://convlab.blob.core.windows.net/convlab-2/mdrg_model.zip
08/02/2020 18:07:01 - INFO - convlab2.util.allennlp_file_utils -   https://convlab.blob.core.windows.net/convlab-2/mdrg_model.zip not found in cache, downloading to /tmp/tmpfwukx3ri
100%|██████████| 21577107/21577107 [00:09<00:00, 2294215.78B/s]
08/02/2020 18:07:12 - INFO - convlab2.util.allennlp_file_utils -   copying /tmp/tmpfwukx3ri to cache at /home/syseo/.convlab2/cache/b0bc758ff68dc79ef5287ddd38b6267f8784df273b2e6f7e496a1e9031c65ca5.ea9a4a5a9034b22be1093ea89deb230956f14487e2c2441b9ee59cef0fc252a2
08/02/2020 18:07:12 - INFO - convlab2.util.allennlp_file_utils -   creating metadata file for /home/syseo/.convlab2/cache/b0bc758ff68dc79ef5287ddd38b6267f8784df273b2e6f7e496a1e9031c65ca5.ea9a4a5a9034b22be1093ea89deb230956f14487e2c2441b9ee59cef0fc252a2
08/02/2020 18:07:12 - INFO - convlab2.util.allennlp_file_utils -   removing temp file /tmp/tmpfwukx3ri
Extracting...
Load from https://convlab.blob.core.windows.net/c

NLU+RuleDST or Word-DST:

In [14]:
# NLU+RuleDST:
sys_nlu = BERTNLU()
# sys_nlu = MILU()
# sys_nlu = SVMNLU()
sys_dst = RuleDST()

# or Word-DST:
# sys_nlu = None
# sys_dst = SUMBT()
# sys_dst = TRADE()
# sys_dst = MDBT()

08/02/2020 18:07:37 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): s3.amazonaws.com:443
08/02/2020 18:07:38 - DEBUG - urllib3.connectionpool -   https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/bert-base-uncased-vocab.txt HTTP/1.1" 200 0
08/02/2020 18:07:38 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/syseo/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
08/02/2020 18:07:38 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): s3.amazonaws.com:443
intent num: 137
tag num: 331
Load from /home/syseo/workspace/ConvLab-2/convlab2/nlu/jointBERT/multiwoz/output/all_context/pytorch_model.bin
bert-base-uncased
08/02/2020 18:07:39 - DEBUG - urllib3.connectionpool -   https://s3.amazonaws.com:443 "HEAD /models.hugging

Policy+NLG or Word-Policy:

In [30]:
# Policy+NLG:
sys_policy = RulePolicy()
# sys_policy = PPOPolicy()
# sys_policy = PGPolicy()
# sys_policy = MLEPolicy()
# sys_policy = GDPLPolicy()
# sys_nlg = TemplateNLG(is_user=False)
sys_nlg = SCLSTM(is_user=False)

# or Word-Policy:
# sys_policy = LaRL()
# sys_policy = HDSA()
# sys_policy = MDRGWordPolicy()
# sys_nlg = None

08/02/2020 18:35:03 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): convlab.blob.core.windows.net:443
Load from https://convlab.blob.core.windows.net/convlab-2/nlg_sclstm_multiwoz.zip
08/02/2020 18:35:03 - DEBUG - urllib3.connectionpool -   https://convlab.blob.core.windows.net:443 "HEAD /convlab-2/nlg_sclstm_multiwoz.zip HTTP/1.1" 200 0
08/02/2020 18:35:03 - INFO - convlab2.util.allennlp_file_utils -   https://convlab.blob.core.windows.net/convlab-2/nlg_sclstm_multiwoz.zip not found in cache, downloading to /tmp/tmpqvri1m1a
08/02/2020 18:35:03 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): convlab.blob.core.windows.net:443
08/02/2020 18:35:04 - DEBUG - urllib3.connectionpool -   https://convlab.blob.core.windows.net:443 "GET /convlab-2/nlg_sclstm_multiwoz.zip HTTP/1.1" 200 18378637
100%|██████████| 18378637/18378637 [00:08<00:00, 2262898.72B/s]
08/02/2020 18:35:12 - INFO - convlab2.util.allennlp_file_utils -   copying /tmp/tmpqvri1m1a t

FileNotFoundError: [Errno 2] No such file or directory: '/home/syseo/workspace/ConvLab-2/convlab2/nlg/sclstm/multiwoz/resource/vocab.txt'

Assemble the Pipeline system agent:

In [16]:
sys_agent = PipelineAgent(sys_nlu, sys_dst, sys_policy, sys_nlg, 'sys')

Or Directly use an end-to-end model:

In [17]:
# sys_agent = Sequicity()
# sys_agent = Damd()

Config an user agent similarly:

In [18]:
user_nlu = BERTNLU()
# user_nlu = MILU()
# user_nlu = SVMNLU()
user_dst = None
user_policy = RulePolicy(character='usr')
# user_policy = UserPolicyVHUS(load_from_zip=True)
user_nlg = TemplateNLG(is_user=True)
# user_nlg = SCLSTM(is_user=True)
user_agent = PipelineAgent(user_nlu, user_dst, user_policy, user_nlg, name='user')

08/02/2020 18:07:42 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): s3.amazonaws.com:443
08/02/2020 18:07:42 - DEBUG - urllib3.connectionpool -   https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/bert-base-uncased-vocab.txt HTTP/1.1" 200 0
08/02/2020 18:07:42 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/syseo/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
08/02/2020 18:07:42 - DEBUG - urllib3.connectionpool -   Starting new HTTPS connection (1): s3.amazonaws.com:443
intent num: 137
tag num: 331
Load from /home/syseo/workspace/ConvLab-2/convlab2/nlu/jointBERT/multiwoz/output/all_context/pytorch_model.bin
bert-base-uncased
08/02/2020 18:07:43 - DEBUG - urllib3.connectionpool -   https://s3.amazonaws.com:443 "HEAD /models.hugging

## Use analysis tool to diagnose the system
We provide an analysis tool presents rich statistics and summarizes common mistakes from simulated dialogues, which facilitates error analysis and
system improvement. The analyzer will generate an HTML report which contains
rich statistics of simulated dialogues. For more information, please refer to `convlab2/util/analysis_tool`.

In [19]:
from convlab2.util.analysis_tool.analyzer import Analyzer

# if sys_nlu!=None, set use_nlu=True to collect more information
analyzer = Analyzer(user_agent=user_agent, dataset='multiwoz')

set_seed(20200131)
analyzer.comprehensive_analyze(sys_agent=sys_agent, model_name='sys_agent', total_dialog=100)

335
08/02/2020 18:08:40 - DEBUG - matplotlib.font_manager -   findfont: score(<Font 'Lohit Gurmukhi' (Lohit-Gurmukhi.ttf) normal normal 400 normal>) = 10.335
08/02/2020 18:08:40 - DEBUG - matplotlib.font_manager -   findfont: score(<Font 'Samyak Devanagari' (Samyak-Devanagari.ttf) normal normal 500 normal>) = 10.24
08/02/2020 18:08:40 - DEBUG - matplotlib.font_manager -   findfont: score(<Font 'Umpush' (Umpush-BoldOblique.ttf) oblique normal 700 normal>) = 11.05
08/02/2020 18:08:40 - DEBUG - matplotlib.font_manager -   findfont: score(<Font 'Waree' (Waree.ttf) normal normal 400 normal>) = 10.335
08/02/2020 18:08:40 - DEBUG - matplotlib.font_manager -   findfont: score(<Font 'KacstTitleL' (KacstTitleL.ttf) normal normal 400 normal>) = 10.335
08/02/2020 18:08:40 - DEBUG - matplotlib.font_manager -   findfont: score(<Font 'Tlwg Mono' (TlwgMono-BoldOblique.ttf) oblique normal 700 normal>) = 11.05
08/02/2020 18:08:40 - DEBUG - matplotlib.font_manager -   findfont: score(<Font 'DejaVu Sans M

(0.8,
 0.72,
 0.7606541606541607,
 0.8741221741221741,
 0.7933313880283577,
 0.8951612903225806,
 17.06)

To compare several models:

In [21]:
set_seed(20200131)
analyzer.compare_models(agent_list=[sys_agent, sys_agent], model_name=['sys_agent1', 'sys_agent2'], total_dialog=100)

hone domain: taxi)
08/02/2020 18:10:17 - DEBUG - root -   Value not found in standard value set: [38559844436] (slot: phone domain: taxi)
08/02/2020 18:10:17 - DEBUG - root -   Value not found in standard value set: [00147824192] (slot: phone domain: taxi)
08/02/2020 18:10:17 - DEBUG - root -   Value not found in standard value set: [95415667871] (slot: phone domain: taxi)
08/02/2020 18:10:17 - DEBUG - root -   Value not found in standard value set: [80603220423] (slot: phone domain: taxi)
08/02/2020 18:10:17 - DEBUG - root -   Value not found in standard value set: [16881767199] (slot: phone domain: taxi)
08/02/2020 18:10:17 - DEBUG - root -   Value not found in standard value set: [62228750300] (slot: phone domain: taxi)
08/02/2020 18:10:18 - DEBUG - root -   Value not found in standard value set: [27333988158] (slot: phone domain: taxi)
dialogue:  89%|████████▉ | 89/100 [00:41<00:03,  3.03it/s]08/02/2020 18:10:18 - DEBUG - root -   Value not found in standard value set: [do n't care